In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [3]:
# 📦 Import Required Modules
from fastapi import APIRouter, Depends, HTTPException, status
from sqlalchemy.orm import Session
from uuid import UUID
from datetime import datetime

from database import get_db
import models, schemas

router = APIRouter(prefix="/trading", tags=["Trading"])

✅ Database connection established successfully.
[NbConvertApp] WARNING | pattern 'database.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to:

/opt/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


[NbConvertApp] WARNING | pattern 'schemas.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Exe

In [4]:
# 📈 Place a New Trade
@router.post("/trade", response_model=schemas.TradeResponse)
def execute_trade(trade: schemas.TradeCreate, db: Session = Depends(get_db)):
    # ✅ Check if User Exists
    user = db.query(models.User).filter(models.User.id == trade.user_id).first()
    if not user:
        raise HTTPException(status_code=404, detail="User not found")

    # 🔢 Check S-Coin Balance
    balance = db.query(models.SCoinBalance).filter(models.SCoinBalance.user_id == trade.user_id).first()
    if not balance or balance.balance < trade.amount:
        raise HTTPException(status_code=400, detail="Insufficient S-Coin balance")

    # 💸 Deduct Balance
    balance.balance -= trade.amount
    db.add(balance)

    # 💾 Create Trade Transaction
    new_trade = models.TradeTransaction(
        user_id=trade.user_id,
        stock_symbol=trade.stock_symbol,
        trade_type=trade.trade_type,
        amount=trade.amount,
        price=trade.price,
        timestamp=datetime.utcnow()
    )
    db.add(new_trade)
    db.commit()
    db.refresh(new_trade)

    return new_trade

In [5]:
# 📜 View Trade History
@router.get("/history/{user_id}", response_model=list[schemas.TradeResponse])
def get_trade_history(user_id: UUID, db: Session = Depends(get_db)):
    trades = db.query(models.TradeTransaction).filter(models.TradeTransaction.user_id == user_id).order_by(models.TradeTransaction.timestamp.desc()).all()
    return trades

In [6]:
# 💼 View Portfolio Balance (S-Coins)
@router.get("/balance/{user_id}", response_model=schemas.SCoinBalanceResponse)
def get_balance(user_id: UUID, db: Session = Depends(get_db)):
    balance = db.query(models.SCoinBalance).filter(models.SCoinBalance.user_id == user_id).first()
    if not balance:
        raise HTTPException(status_code=404, detail="Balance not found")
    return balance

In [16]:
# # !jupyter nbconvert --to script ~/Desktop/stock_trading_sim/app/routes/trading.ipynb

try:
    get_ipython().system('jupyter nbconvert --to script trading.ipynb')
except:
    pass


[NbConvertApp] Converting notebook trading.ipynb to script
[NbConvertApp] Writing 2483 bytes to trading.py
